In [1]:
include("optim_lib.jl")
include("lib_func.jl")
using Optim, DataFrames, CSV

# Load the dataframe from the CSV file
df_2d = CSV.read("data/camel_3_d6.csv", DataFrame)

# Constants and Parameters
d1, d2, ds = 6, 6, 1  # Degree range and step
const n, a, b = 4, 5, 1 
const C = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alph = .1 , 2 / 10  # Sampling parameters
f = camel_3_by_3 # Objective function


camel_3_by_3 (generic function with 1 method)

In [2]:
coeffs_poly_approx = main_gen(f, n, d1, d2, ds, delta, alph, C, 0.2)

dimension Vector space: 210
sample size: 50625


1-element Vector{Vector{Float64}}:
 [1210.4166666666672, 1.6834897718883213e-13, 917.5781249999999, 1.679942189219675e-13, 406.24999999999966, 1.67073233507405e-13, 81.3802083333334, -3.8967397578097976e-14, 25.00000000000001, -4.181649841586846e-15  …  -2.252606760942147e-13, 2.026151873156143e-13, -5.020610561616735e-15, -2.9245770182698234e-15, -5.187826809211868e-13, -7.220185913010137e-13, 1.5398987506649635e-14, -5.4912564044645926e-14, 4.913153182680428e-13, -2.7190594519747354e-12]

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [3]:
using DynamicPolynomials, MultivariatePolynomials, HomotopyContinuation, ProgressLogging, DataFrames, Optim

include("hom_solve.jl") # Include the homotopy solver and main function
vars = @polyvar(x[1:n]) # Define polynomial ring 
h_x, h_y, h_z, h_t, col = main_4d(n, d1, d2, ds, coeffs_poly_approx) # main_4d is in hom_solve.jl
df = DataFrame(x=C * h_x, y=C * h_y, z=C * h_z, t=C * h_t, col=col)
df[!, :val_f] = [f([df.x[i], df.y[i], df.z[i], df.t[i]]) for i in 1:nrow(df)]
df[!, :total_distance] = zeros(nrow(df))
df[!, :dist_to_loc_min] = zeros(Float64, nrow(df))
df[!, :steps] = zeros(Int, nrow(df))
df[!, :converged] = falses(nrow(df))

sorted_df = sort(df, :val_f)

Computing mixed cells... 2    Time: 0:00:00
Computing mixed cells... 63    Time: 0:00:01
  mixed_volume:  625


Tracking 625 paths...   0%|▏                            |  ETA: 1:34:50



Tracking 625 paths...  30%|████████▊                    |  ETA: 0:00:48



Tracking 625 paths...  34%|██████████                   |  ETA: 0:00:40



Tracking 625 paths...  39%|███████████▏                 |  ETA: 0:00:34



Tracking 625 paths...  43%|████████████▍                |  ETA: 0:00:28



Tracking 625 paths...  47%|█████████████▊               |  ETA: 0:00:24



Tracking 625 paths...  52%|███████████████              |  ETA: 0:00:20



Tracking 625 paths...  56%|████████████████▍            |  ETA: 0:00:17



Tracking 625 paths...  60%|█████████████████▌           |  ETA: 0:00:14



Tracking 625 paths...  64%|██████████████████▋          |  ETA: 0:00:12



Tracking 625 paths...  68%|███████████████████▉         |  ETA: 0:00:10



Tracking 625 paths...  73%|█████████████████████▏       |  ETA: 0:00:08



Tracking 625 paths...  78%|██████████████████████▌      |  ETA: 0:00:07



Tracking 625 paths...  82

Row,x,y,z,t,col,val_f,total_distance,dist_to_loc_min,steps,converged
,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Int64,Bool
1,1.54971e-14,-1.23198e-13,3.8878e-13,1.00232e-13,6,3.65063e-25,0.0,0.0,0,false
2,2.14315e-14,-1.36495e-13,1.74755,-0.873776,6,0.298638,0.0,0.0,0,false
3,-1.74755,0.873776,3.93355e-13,6.56749e-14,6,0.298638,0.0,0.0,0,false
4,9.78055e-15,-1.17944e-13,-1.74755,0.873776,6,0.298638,0.0,0.0,0,false
5,1.74755,-0.873776,4.0445e-13,6.66781e-14,6,0.298638,0.0,0.0,0,false
6,-1.74755,0.873776,-1.74755,0.873776,6,0.597277,0.0,0.0,0,false
7,-1.74755,0.873776,1.74755,-0.873776,6,0.597277,0.0,0.0,0,false
8,1.74755,-0.873776,1.74755,-0.873776,6,0.597277,0.0,0.0,0,false
9,1.74755,-0.873776,-1.74755,0.873776,6,0.597277,0.0,0.0,0,false


Now we want to compute the distance between the critical points of the 2d function. We want to take the tensor product of coordinates of critical points recovered from the 2D case. Add a column for minimal distance of 2 first coordinates to original critical point, same for 2nd column. 

The true critical points are stored in df_2d. Need to split to distance to minima first coordinate and second set of coordinates ten combine them.


In [26]:
for i in 1:nrow(df)
    distances_1 = [norm([df.x[i], df.y[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    distances_2 = [norm([df.z[i], df.t[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    min_value_1, min_index_1 = findmin(distances_1)
    min_value_2, min_index_2 = findmin(distances_2)    
    df.total_distance[i] = norm([df.x[i], df.y[i], df.z[i], df.t[i]] - [df_2d.x[min_index_1], df_2d.y[min_index_1], df_2d.x[min_index_2], df_2d.y[min_index_2]])
end
sorted_df = sort(df, :total_distance)

Row,x,y,z,t,col,val_f,total_distance,dist_to_loc_min,steps,converged
,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Int64,Bool
1,-1.74755,0.873776,3.93355e-13,6.56749e-14,6,0.298638,4.40152e-13,0.0,0,true
2,-1.07054,0.535271,3.89238e-13,8.63535e-14,6,0.877362,4.59623e-13,0.0,0,true
3,1.54971e-14,-1.23198e-13,3.8878e-13,1.00232e-13,6,3.65063e-25,4.64413e-13,0.0,0,true
4,1.07054,-0.535271,3.96089e-13,8.82379e-14,6,0.877362,5.37214e-13,0.0,0,true
5,1.74755,-0.873776,4.0445e-13,6.66781e-14,6,0.298638,6.59417e-13,0.0,0,true
6,1.89846e-14,-1.29946e-13,1.07054,-0.535271,6,0.877362,7.96295e-13,0.0,0,true
7,-1.07054,0.535271,1.07054,-0.535271,6,1.75472,8.08604e-13,0.0,0,true
8,-1.74755,0.873776,1.07054,-0.535271,6,1.176,8.09302e-13,0.0,0,true
9,1.07054,-0.535271,1.07054,-0.535271,6,1.75472,8.49068e-13,0.0,0,true


In [28]:
using LinearAlgebra
# Precompute df_2d coordinates
df_2d_coords = [df_2d.x df_2d.y]

# Create new columns for results
df.total_distance = Vector{Float64}(undef, nrow(df))

for i in 1:nrow(df)
    # Current coordinates
    coords_1 = [df.x[i], df.y[i]]
    coords_2 = [df.z[i], df.t[i]]

    # Compute distances to all points in df_2d
    distances_1 = vec(norm.(eachrow(df_2d_coords .- coords_1')))
    distances_2 = vec(norm.(eachrow(df_2d_coords .- coords_2')))

    # Find minimum distances and corresponding indices
    min_value_1, min_index_1 = findmin(distances_1)
    min_value_2, min_index_2 = findmin(distances_2)

    # Compute total distance
    min_coords_1 = df_2d_coords[min_index_1, :]
    min_coords_2 = df_2d_coords[min_index_2, :]
    df.total_distance[i] = norm([df.x[i], df.y[i], df.z[i], df.t[i]] - [min_coords_1; min_coords_2])
end

# Display the dataframe
sorted_df = sort(df, :val_f)
println(sorted_df)

25×10 DataFrame
 Row │ x             y             z             t             col    val_f        total_distance  dist_to_loc_min  steps  converged 
     │ Float64       Float64       Float64       Float64       Int64  Float64      Float64         Float64          Int64  Bool      
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │  1.54971e-14  -1.23198e-13   3.8878e-13    1.00232e-13      6  3.65063e-25     4.64413e-13              0.0      0       true
   2 │  2.14315e-14  -1.36495e-13   1.74755      -0.873776         6  0.298638        1.34488e-12              0.0      0       true
   3 │ -1.74755       0.873776      3.93355e-13   6.56749e-14      6  0.298638        4.40152e-13              0.0      0       true
   4 │  9.78055e-15  -1.17944e-13  -1.74755       0.873776         6  0.298638        1.63269e-12              0.0      0       true
   5 │  1.74755      -0.873776      4.0445e-13    

In [21]:
for i in 1:nrow(df)
    # println("Optimizing for point $i")
    x0 = [df.x[i], df.y[i], df.z[i], df.t[i]]
    #  + 0.2 * randn(Float64, 4)
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)
    distance = norm(x0 - minimizer)
    df.dist_to_loc_min[i] = distance
    df.steps[i] = steps
    df.converged[i] = converged
    # summary(res)
end
print(df)

Iter     Function value   Gradient norm 
     0     5.972769e-01     1.164545e-09
 * time: 5.984306335449219e-5
Iter     Function value   Gradient norm 
     0     1.176000e+00     1.279950e-09
 * time: 5.507469177246094e-5
Iter     Function value   Gradient norm 
     0     1.176000e+00     1.154054e-09
 * time: 2.5987625122070312e-5
Iter     Function value   Gradient norm 
     0     1.754723e+00     3.425229e-11
 * time: 3.1948089599609375e-5
Iter     Function value   Gradient norm 
     0     1.754723e+00     3.425229e-11
 * time: 2.4080276489257812e-5
Iter     Function value   Gradient norm 
     0     1.176000e+00     1.154054e-09
 * time: 2.384185791015625e-5
Iter     Function value   Gradient norm 
     0     8.773616e-01     1.712615e-11
 * time: 1.7881393432617188e-5
Iter     Function value   Gradient norm 
     0     5.972769e-01     1.196019e-09
 * time: 2.193450927734375e-5
Iter     Function value   Gradient norm 
     0     1.176000e+00     1.154054e-09
 * time: 1.8119812